In [1]:
import pandas as pd
import numpy as np
import json
from altair import datum
import warnings
import altair as alt
warnings.filterwarnings("ignore")

### Loading EV Registration Data

In [2]:
ev_reg_data2 = pd.read_csv('ev_reg_data.csv')
ev_reg_data2 = ev_reg_data2[ev_reg_data2["State"] != 'District of Columbia'] #dropping district of columbia
ev_reg_data2

,Year,State,Total EV registration
0,2016,Alabama,1400
1,2016,Alaska,400
2,2016,Arizona,9100
3,2016,Arkansas,700
4,2016,California,258200
...,...,...,...
352,2022,Virginia,78300
353,2022,Washington,135500
354,2022,West Virginia,3300
355,2022,Wisconsin,25700


### Loading Cummulative Station Data

In [3]:
stations_data = pd.read_csv('df_stations_cum.csv')
stations_data = stations_data.rename(columns={'Available Stations': 'Total Stations'})
stations_data = stations_data[stations_data["State"] != 'District Of Columbia'] # dropping District of Columbia
stations_final = stations_data.drop(['Newly Opened Stations'], axis = 1) #dropping newly opened stations column (only want cum.)
stations_final

,Year,State,Total Stations
0,2016,Alabama,89
1,2017,Alabama,123
2,2018,Alabama,139
3,2019,Alabama,157
4,2020,Alabama,189
...,...,...,...
352,2018,Wyoming,49
353,2019,Wyoming,54
354,2020,Wyoming,56
355,2021,Wyoming,65


### Creating Combined Dataframes with "All' Option

In [4]:
# Creating a new dataframe that sums registrations across all states for each year
all_states_data = ev_reg_data2.groupby('Year', as_index=False).agg({'Total EV registration': 'sum'})
all_states_data['State'] = 'All'

# Appending the "All" data to the original ev_reg_data2 dataframe
ev_data_combined = pd.concat([ev_reg_data2, all_states_data], ignore_index=True)

# Aggregate the number of stations by year
sum_stations_by_year =stations_final.groupby('Year',as_index = False)['Total Stations'].sum().reset_index()
sum_stations_by_year['State'] = 'All'

# Appending the "All" data to the original dataframe
stations_combined = pd.concat([stations_final, sum_stations_by_year], ignore_index = True)

### Creating Visualization

In [5]:
# Creating a drop-down selection for filtering by state, including the "All" option
state_selection = alt.selection_point(
    fields=['State'],
    bind=alt.binding_select(options=['All'] + ev_reg_data2['State'].unique().tolist(), name='Select State: '),
    name='State',
    value = 'All'
)

# Creating the line chart for ev_registrations data
line_chart = alt.Chart(ev_data_combined).mark_line().encode(
    x=alt.X('Year:O', title='Year',axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Total EV registration:Q', title='Number of Registrations', axis=alt.Axis(titleColor='orange')),
    color=alt.value("orange"),
    tooltip=[
        alt.Tooltip("Year:O"),
        alt.Tooltip("State:N"),
        alt.Tooltip("Total EV registration:Q",format=",.0f")
    ]
).transform_filter(
    state_selection)

# Creating the point chart for ev_registrations
point_chart = alt.Chart(ev_data_combined).mark_point(filled=True,size=100).encode(
    x=alt.X('Year:O', title='Year',axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Total EV registration:Q', title=None, axis=alt.Axis(titleColor='orange')),
    color=alt.value("orange"),
    tooltip=[
        alt.Tooltip("Year:O"),
        alt.Tooltip("State:N"),
        alt.Tooltip("Total EV registration:Q",format=",.0f")
    ]
).transform_filter(
    state_selection)

# creating bar chart for stations_data
stations = alt.Chart(stations_combined).mark_bar(size = 60
).encode(
    x=alt.X('Year:O', title='Year',axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Total Stations:Q', title='Total number of Stations', axis=alt.Axis(titleColor='purple')),
    color=alt.value("purple"),
    tooltip=[
        alt.Tooltip("Year:O"),
        alt.Tooltip("State:N"),
        alt.Tooltip("Total Stations:Q",format=",.0f")
    ]
).transform_filter(
    state_selection)

#Combined Chart
combined_chart2 = alt.layer(stations, line_chart, point_chart
).resolve_scale(
    y='independent'
).add_selection(
    state_selection
).properties(
    width=800,
    height=600,
    title= alt.TitleParams(
            text={'expr': f'"Electric Vehicle Registrations and Number of Charging Stations by Year: " + {state_selection.name}.State'},
            fontSize=20
    )
).configure_view(
    strokeWidth=0
).configure_axis(
    labelFontSize=15, 
    titleFontSize=20, 
    grid=True
).configure_title(
    fontSize=25 
).configure_legend(
    labelFontSize=15,  
    titleFontSize=20,  
    labelLimit=0
)

combined_chart2


alt.LayerChart(...)

In [6]:
combined_chart2.save("car_registration.html")